# SESIÓN 1: Asincronía y Consumo de APIs Externas

**Objetivos de aprendizaje:**
- Comprender el paradigma asíncrono y su beneficio en producción
- Implementar endpoints async en FastAPI
- Consumir APIs externas de forma no bloqueante
- Utilizar BackgroundTasks para operaciones diferidas

## Requisitos del entorno

Este notebook debe ejecutarse en:
- VSCode con extensión Jupyter
- Entorno virtual Python activado
- Python 3.8 o superior

## CONFIGURACIÓN DEL ENTORNO

### Paso 1: Verificación de Python

Antes de comenzar, verificamos la versión de Python disponible en nuestro entorno.

In [ ]:
import sys
print(f"Python: {sys.version}")

# Verificar que estamos en Python 3.8+
assert sys.version_info >= (3, 8), "Se requiere Python 3.8 o superior"
print("\nVerificación exitosa. Python versión compatible.")

### Paso 2: Instalación de dependencias

Instalamos las librerías necesarias para esta sesión. Este comando utiliza pip sin necesidad de permisos de administrador.

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 httpx==0.27.0 uvicorn[standard]==0.32.0 -q

print("Dependencias instaladas correctamente")

### Paso 3: Verificación de versiones instaladas

Comprobamos que las librerías se instalaron correctamente y vemos sus versiones.

In [ ]:
import fastapi
import httpx
import uvicorn

print("Versiones instaladas:")
print(f"FastAPI: {fastapi.__version__}")
print(f"HTTPX: {httpx.__version__}")
print(f"Uvicorn: {uvicorn.__version__}")
print("\nTodas las dependencias están listas.")

### Paso 4: Importaciones globales

Importamos todos los módulos que utilizaremos a lo largo de esta sesión.

In [ ]:
# Módulos estándar de Python
import asyncio
import time
from datetime import datetime
import threading

# FastAPI y componentes
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.testclient import TestClient
from pydantic import BaseModel, HttpUrl

# Cliente HTTP asíncrono
import httpx

print("Todas las importaciones completadas exitosamente")

## BLOQUE 1: Fundamentos de la Asincronía

### Contexto histórico: De Python síncrono a asíncrono

Antes de Python 3.5, todo el código Python era síncrono (bloqueante):
- Si llamabas a una API externa, tu programa esperaba la respuesta sin hacer nada más
- Si procesabas 10 requests, los atendías uno a uno

Python 3.5 (2015) introdujo `async/await`, inspirándose en JavaScript y C#:
- Ahora podemos iniciar una tarea y continuar con otra mientras esperamos
- Un solo proceso puede manejar miles de conexiones concurrentes

### Conceptos clave

**Event Loop (Bucle de eventos):**
- Es el "director de orquesta" que gestiona todas las tareas asíncronas
- Cuando una tarea espera (I/O, red, base de datos), el event loop pasa a ejecutar otra
- FastAPI gestiona el event loop automáticamente

**Coroutine (Corutina):**
- Una función definida con `async def`
- Puede "pausarse" con `await` y ceder control al event loop
- Cuando se reanuda, continúa desde donde se pausó

**Analogía del restaurante:**
```
MODELO SÍNCRONO (Bloqueante):
Mesero toma orden → Va a cocina → Espera el plato → Lo trae → Toma siguiente orden
(Si hay 10 mesas, la última espera mucho tiempo)

MODELO ASÍNCRONO (No bloqueante):
Mesero toma orden mesa 1 → Pasa a cocina → Toma orden mesa 2 → Pasa a cocina → ...
Cuando un plato está listo, lo entrega y continúa con otras mesas
(10 mesas pueden ser atendidas casi simultáneamente por un solo mesero)
```

### Demostración: Código síncrono vs asíncrono

Vamos a simular una operación que tarda 2 segundos (como una llamada a una base de datos o API externa).
Compararemos el tiempo total para procesar 3 operaciones de forma síncrona y asíncrona.

In [ ]:
# VERSIÓN SÍNCRONA (Bloqueante)
# Ya importamos time en la configuración inicial

def operacion_lenta_sincrona(numero):
    print(f"Iniciando operación {numero}...")
    time.sleep(2)  # Simula operación que tarda 2 segundos
    print(f"Operación {numero} completada")
    return f"Resultado {numero}"

inicio = time.time()

# Ejecutamos 3 operaciones secuencialmente
resultado_1 = operacion_lenta_sincrona(1)
resultado_2 = operacion_lenta_sincrona(2)
resultado_3 = operacion_lenta_sincrona(3)

tiempo_total = time.time() - inicio
print(f"\nTiempo total síncrono: {tiempo_total:.2f} segundos")

In [ ]:
# VERSIÓN ASÍNCRONA (No bloqueante)
# asyncio también fue importado en la configuración inicial

async def operacion_lenta_asincrona(numero):
    print(f"Iniciando operación {numero}...")
    await asyncio.sleep(2)  # Simula operación que tarda 2 segundos (no bloqueante)
    print(f"Operación {numero} completada")
    return f"Resultado {numero}"

inicio = time.time()

# Ejecutamos 3 operaciones concurrentemente
resultados = await asyncio.gather(
    operacion_lenta_asincrona(1),
    operacion_lenta_asincrona(2),
    operacion_lenta_asincrona(3)
)

tiempo_total = time.time() - inicio
print(f"\nTiempo total asíncrono: {tiempo_total:.2f} segundos")
print(f"Resultados: {resultados}")

**Observa los resultados:**
- Versión síncrona: aproximadamente 6 segundos (2 + 2 + 2)
- Versión asíncrona: aproximadamente 2 segundos (las 3 operaciones se ejecutan "en paralelo")

**Nota importante:** La asincronía NO utiliza múltiples CPUs (eso sería paralelismo con multiprocessing). La asincronía es útil cuando el tiempo se pierde esperando I/O (red, disco, base de datos), no para cálculos intensivos.

### MICRO-RETO 1: Primera corutina

Crea una corutina llamada `saludar_con_delay` que:
1. Reciba un nombre como parámetro
2. Espere 1 segundo de forma asíncrona
3. Retorne el mensaje "Hola, {nombre}"

Luego ejecútala 3 veces concurrentemente y mide el tiempo total.

In [ ]:
# TODO: Tu código aquí
# Recuerda: asyncio y time ya están importados globalmente
import asyncio
import time

async def saludar_con_delay(nombre):
    # Usamos el await asyncio.sleep() para no bloquear el hilo principal
    await asyncio.sleep(1)
    return f"Hola {nombre}"

# Ejecuta 3 saludos concurrentemente y mide el tiempo
inicio = time.time()
# Tu código aquí
await asyncio.gather(
    saludar_con_delay("Ana"),
    saludar_con_delay("Carlos"),
    saludar_con_delay("María")
)
tiempo_total = time.time() - inicio

# Verifica que el tiempo total sea cercano a 1 segundo (no 3)
assert tiempo_total < 1.5, "Las operaciones no se ejecutaron concurrentemente"
print("MICRO-RETO 1 COMPLETADO")

## BLOQUE 2: async def en FastAPI

### Cuándo usar async def vs def en endpoints

FastAPI te permite definir endpoints de dos formas:

```python
@app.get("/sincrono")
def endpoint_sincrono():
    return {"mensaje": "Soy síncrono"}

@app.get("/asincrono")
async def endpoint_asincrono():
    return {"mensaje": "Soy asíncrono"}
```

**Regla de oro:**
- Usa `async def` cuando tu endpoint realice operaciones I/O: llamadas a BD, APIs externas, lectura de archivos
- Usa `def` cuando tu endpoint solo procese datos en memoria o realice cálculos

**¿Qué pasa internamente?**
- Endpoints `def`: FastAPI los ejecuta en un thread pool (máximo 40 threads por defecto)
- Endpoints `async def`: FastAPI los ejecuta en el event loop (puede manejar miles concurrentemente)

**Cambio desde versiones antiguas:**
- Antes de FastAPI 0.61 (2020), no había diferencia práctica entre `def` y `async def`
- Desde 0.61+, usar `async def` correctamente puede mejorar el rendimiento hasta 10x en escenarios con muchas conexiones simultáneas

### Ejemplo: Endpoint que simula consulta a base de datos

Vamos a crear un endpoint que simula una consulta lenta a base de datos. Compararemos las versiones síncrona y asíncrona.

Nota: FastAPI, asyncio y time ya fueron importados en la configuración inicial.

In [ ]:
# Creamos una nueva instancia de FastAPI para este ejemplo
app = FastAPI()

# VERSIÓN SÍNCRONA
@app.get("/usuarios/sincrono/{usuario_id}")
def obtener_usuario_sincrono(usuario_id: int):
    # Simula consulta a BD que tarda 2 segundos
    time.sleep(2)
    return {
        "id": usuario_id,
        "nombre": f"Usuario {usuario_id}",
        "tipo": "respuesta síncrona"
    }

# VERSIÓN ASÍNCRONA
@app.get("/usuarios/asincrono/{usuario_id}")
async def obtener_usuario_asincrono(usuario_id: int):
    # Simula consulta a BD que tarda 2 segundos (no bloqueante)
    await asyncio.sleep(2)
    return {
        "id": usuario_id,
        "nombre": f"Usuario {usuario_id}",
        "tipo": "respuesta asíncrona"
    }

print("Endpoints creados. Para probarlos:")
print("uvicorn nombre_archivo:app --reload")

**¿Qué diferencia hay en producción?**

Imagina que llegan 10 requests simultáneos:

- Endpoint síncrono: Atenderá máximo 40 requests a la vez (límite del thread pool). Si hay más, esperan en cola.
- Endpoint asíncrono: Puede atender miles de requests simultáneos sin problema.

**Ejemplo real de beneficio:**
- Empresa X tenía una API con endpoints síncronos
- Con 100 usuarios concurrentes, el servidor alcanzaba el límite
- Cambió a async def en endpoints que consultaban BD
- Ahora soporta 5,000 usuarios concurrentes en el mismo servidor

### MICRO-RETO 2: Tu primer endpoint asíncrono

Crea un endpoint POST `/calcular` que:
1. Reciba un JSON con dos números: `{"a": 5, "b": 3}`
2. Simule una operación lenta (await asyncio.sleep(1))
3. Retorne la suma, resta y multiplicación de ambos números

In [ ]:
import uvicorn
import threading
import time
# TODO: Tu código aquí
# Recuerda: FastAPI, BaseModel, asyncio y TestClient ya están importados

app = FastAPI()

class Numeros(BaseModel):
    a: int
    b: int

# Implementa tu endpoint aquí

# Verificación (no modificar)
client = TestClient(app)
respuesta = client.post("/calcular", json={"a": 10, "b": 5})
assert respuesta.status_code == 200
assert respuesta.json()["suma"] == 15
print("MICRO-RETO 2 COMPLETADO")

# Iniciar servidor en puerto 8000
def run_server_8000():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="warning")

server_thread = threading.Thread(target=run_server_8000, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8000")
print(f"📚 Documentación interactiva: http://127.0.0.1:8000/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8000/redoc")


## BLOQUE 3: Consumo de APIs Externas con httpx

### Por qué httpx en lugar de requests

Hasta hace poco, la librería estándar para llamadas HTTP en Python era `requests`:

```python
import requests
respuesta = requests.get("https://api.ejemplo.com/datos")
```

Pero `requests` es **completamente síncrona**. Si la usas en un endpoint `async def`, bloqueas el event loop.

**httpx** es la alternativa moderna que soporta tanto sync como async:
```python
import httpx

# Síncrono (compatible con requests)
respuesta = httpx.get("https://api.ejemplo.com/datos")

# Asíncrono (para usar con await)
async with httpx.AsyncClient() as cliente:
    respuesta = await cliente.get("https://api.ejemplo.com/datos")
```

**Instalación:**
```bash
pip install httpx
```

**Ventajas de httpx sobre requests:**
- Soporta HTTP/2 (más rápido)
- API asíncrona nativa
- Timeouts más configurables
- Mejor manejo de conexiones persistentes

### Ejemplo: Consumir una API pública

Vamos a consumir la API pública de JSONPlaceholder (fake REST API para testing).

Nota: httpx ya fue instalado e importado en la configuración inicial.

In [ ]:
# Llamada asíncrona simple usando httpx
async def obtener_post(post_id: int):
    url = f"https://jsonplaceholder.typicode.com/posts/{post_id}"
    
    async with httpx.AsyncClient() as cliente:
        respuesta = await cliente.get(url)
        return respuesta.json()

# Prueba
post = await obtener_post(1)
print(f"Título: {post['title']}")
print(f"Cuerpo: {post['body'][:50]}...")

**Desglose del código:**

1. `async with httpx.AsyncClient() as cliente:` - Crea un cliente HTTP reutilizable. El `async with` asegura que se cierre correctamente.

2. `await cliente.get(url)` - Realiza la petición GET de forma asíncrona. Mientras espera la respuesta, el event loop puede ejecutar otras tareas.

3. `.json()` - Convierte la respuesta a un diccionario Python.

### Manejo de timeouts y errores

En producción, las APIs externas pueden fallar o tardar demasiado. Debemos manejar estos casos.

In [ ]:
# httpx y HTTPException ya están importados globalmente

async def obtener_post_seguro(post_id: int):
    url = f"https://jsonplaceholder.typicode.com/posts/{post_id}"
    
    try:
        async with httpx.AsyncClient(timeout=5.0) as cliente:  # Timeout de 5 segundos
            respuesta = await cliente.get(url)
            respuesta.raise_for_status()  # Lanza excepción si status != 2xx
            return respuesta.json()
            
    except httpx.TimeoutException:
        raise HTTPException(
            status_code=504,
            detail="La API externa tardó demasiado en responder"
        )
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Error en API externa: {e}"
        )
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error inesperado: {str(e)}"
        )

# Prueba
post = await obtener_post_seguro(1)
print("Post obtenido correctamente")

### Endpoint completo con API externa

Ahora integramos todo en un endpoint de FastAPI.

In [ ]:
import uvicorn
import threading
import time
# Creamos una nueva instancia para este ejemplo
app = FastAPI()

@app.get("/posts/{post_id}")
async def obtener_post_externo(post_id: int):
    """
    Obtiene un post desde la API externa JSONPlaceholder.
    """
    url = f"https://jsonplaceholder.typicode.com/posts/{post_id}"
    
    try:
        async with httpx.AsyncClient(timeout=5.0) as cliente:
            respuesta = await cliente.get(url)
            respuesta.raise_for_status()
            datos = respuesta.json()
            
            # Enriquecemos la respuesta con información adicional
            return {
                "fuente": "jsonplaceholder.typicode.com",
                "post_id": post_id,
                "titulo": datos["title"],
                "contenido": datos["body"]
            }
            
    except httpx.TimeoutException:
        raise HTTPException(status_code=504, detail="Timeout al consultar API externa")
    except httpx.HTTPStatusError:
        raise HTTPException(status_code=404, detail="Post no encontrado")

print("Endpoint /posts/{post_id} creado")

# Iniciar servidor en puerto 8001
def run_server_8001():
    uvicorn.run(app, host="127.0.0.1", port=8001, log_level="warning")

server_thread = threading.Thread(target=run_server_8001, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8001")
print(f"📚 Documentación interactiva: http://127.0.0.1:8001/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8001/redoc")


### MICRO-RETO 3: Consumir API de usuarios

Crea un endpoint GET `/usuarios/{usuario_id}` que:
1. Consuma la API: `https://jsonplaceholder.typicode.com/users/{usuario_id}`
2. Maneje timeout de 3 segundos
3. Retorne solo el nombre, email y teléfono del usuario
4. Si el usuario no existe (404), devuelve HTTPException apropiada

In [ ]:
import uvicorn
import threading
import time
# TODO: Tu código aquí
# Recuerda: FastAPI, HTTPException, httpx y TestClient ya están importados

app = FastAPI()

# Implementa tu endpoint aquí

# Verificación (no modificar)
client = TestClient(app)
respuesta = client.get("/usuarios/1")
assert respuesta.status_code == 200
datos = respuesta.json()
assert "nombre" in datos
assert "email" in datos
assert "telefono" in datos
print("MICRO-RETO 3 COMPLETADO")

# Iniciar servidor en puerto 8002
def run_server_8002():
    uvicorn.run(app, host="127.0.0.1", port=8002, log_level="warning")

server_thread = threading.Thread(target=run_server_8002, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8002")
print(f"📚 Documentación interactiva: http://127.0.0.1:8002/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8002/redoc")


## BLOQUE 4: BackgroundTasks para Operaciones Diferidas

### Qué son las BackgroundTasks

A veces queremos que un endpoint responda rápido al usuario, pero necesitamos hacer tareas adicionales que pueden tardar:
- Enviar un email
- Procesar un archivo
- Escribir logs detallados
- Limpiar archivos temporales

**Antes (forma incorrecta):**
```python
@app.post("/crear-cuenta")
async def crear_cuenta(datos: Usuario):
    # 1. Guardar usuario en BD (200ms)
    guardar_en_bd(datos)
    
    # 2. Enviar email de bienvenida (2 segundos)
    enviar_email(datos.email)
    
    return {"mensaje": "Cuenta creada"}  # Usuario espera 2.2 segundos
```

**Ahora (forma correcta):**
```python
@app.post("/crear-cuenta")
async def crear_cuenta(datos: Usuario, background_tasks: BackgroundTasks):
    # 1. Guardar usuario en BD (200ms)
    guardar_en_bd(datos)
    
    # 2. Programar el email para después
    background_tasks.add_task(enviar_email, datos.email)
    
    return {"mensaje": "Cuenta creada"}  # Usuario espera solo 200ms
```

### Ejemplo: Endpoint con tarea en background

Vamos a crear un endpoint que simula procesar una orden y enviar una notificación.

Nota: FastAPI, BackgroundTasks, asyncio y time ya están importados globalmente.

In [ ]:
import uvicorn
import threading
import time
app = FastAPI()

# Esta función se ejecutará en background
async def enviar_notificacion(email: str, mensaje: str):
    print(f"Iniciando envío de notificación a {email}...")
    await asyncio.sleep(3)  # Simula tiempo de envío
    print(f"Notificación enviada a {email}: {mensaje}")

@app.post("/procesar-orden")
async def procesar_orden(
    producto: str,
    email: str,
    background_tasks: BackgroundTasks
):
    # Tarea principal: procesar orden (rápido)
    orden_id = f"ORD-{int(time.time())}"
    print(f"Orden {orden_id} procesada")
    
    # Tarea en background: enviar notificación (lento)
    background_tasks.add_task(
        enviar_notificacion,
        email,
        f"Tu orden {orden_id} de {producto} ha sido procesada"
    )
    
    # Respuesta inmediata al usuario
    return {
        "orden_id": orden_id,
        "mensaje": "Orden procesada. Te enviaremos un email de confirmación."
    }

print("Endpoint /procesar-orden creado")

# Iniciar servidor en puerto 8003
def run_server_8003():
    uvicorn.run(app, host="127.0.0.1", port=8003, log_level="warning")

server_thread = threading.Thread(target=run_server_8003, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8003")
print(f"📚 Documentación interactiva: http://127.0.0.1:8003/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8003/redoc")


**Puntos clave:**

1. La función de background puede ser `async def` o `def` normal
2. Se ejecuta DESPUÉS de enviar la respuesta al usuario
3. Si falla, no afecta la respuesta (pero debes implementar logging para detectarlo)
4. Múltiples tareas pueden agregarse: `background_tasks.add_task(tarea1); background_tasks.add_task(tarea2)`

### Casos de uso reales

**1. Sistema de registro de usuarios:**
```python
@app.post("/registro")
async def registro(datos: Usuario, background_tasks: BackgroundTasks):
    crear_usuario_bd(datos)
    background_tasks.add_task(enviar_email_bienvenida, datos.email)
    background_tasks.add_task(registrar_evento_analytics, "nuevo_usuario")
    return {"mensaje": "Usuario creado"}
```

**2. Procesamiento de archivos:**
```python
@app.post("/subir-imagen")
async def subir_imagen(archivo: UploadFile, background_tasks: BackgroundTasks):
    ruta = guardar_temporalmente(archivo)
    background_tasks.add_task(redimensionar_imagen, ruta)
    background_tasks.add_task(generar_thumbnail, ruta)
    return {"mensaje": "Imagen recibida, procesando..."}
```

**3. Limpieza de archivos temporales:**
```python
@app.get("/reporte-pdf/{reporte_id}")
async def generar_reporte(reporte_id: str, background_tasks: BackgroundTasks):
    archivo_pdf = crear_pdf(reporte_id)
    
    # Eliminar el PDF después de 1 hora
    background_tasks.add_task(eliminar_archivo_despues_de, archivo_pdf, 3600)
    
    return FileResponse(archivo_pdf)
```

### MICRO-RETO 4: Registro con notificación

Crea un endpoint POST `/registrar-usuario` que:
1. Reciba un JSON con `{"nombre": "...", "email": "..."}`
2. Simule guardar el usuario (print simple)
3. Agregue una tarea en background que simule enviar email (await asyncio.sleep(2) + print)
4. Retorne inmediatamente un mensaje de confirmación

In [ ]:
import uvicorn
import threading
import time
# TODO: Tu código aquí
# Recuerda: FastAPI, BackgroundTasks, BaseModel, asyncio y TestClient ya están importados

app = FastAPI()

class Usuario(BaseModel):
    nombre: str
    email: str

# Implementa tu función de background y endpoint aquí

# Verificación (no modificar)
client = TestClient(app)
respuesta = client.post("/registrar-usuario", json={"nombre": "Ana", "email": "ana@test.com"})
assert respuesta.status_code == 200
assert "confirmacion" in respuesta.json() or "mensaje" in respuesta.json()
print("MICRO-RETO 4 COMPLETADO")

# Iniciar servidor en puerto 8004
def run_server_8004():
    uvicorn.run(app, host="127.0.0.1", port=8004, log_level="warning")

server_thread = threading.Thread(target=run_server_8004, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8004")
print(f"📚 Documentación interactiva: http://127.0.0.1:8004/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8004/redoc")


## BLOQUE 5: Caso Integrador Completo

### Arquitectura del sistema

Vamos a construir un endpoint que combine todos los conceptos:

```
Usuario hace POST /analizar-repositorio
    ↓
1. Endpoint recibe URL de GitHub (async def)
    ↓
2. Consulta API de GitHub para obtener info del repo (httpx async)
    ↓
3. Responde inmediatamente al usuario
    ↓
4. En background: analiza issues y genera reporte (BackgroundTasks)
```

### Implementación del sistema completo

Este es un ejemplo realista que podrías encontrar en producción.

Nota: Todas las librerías necesarias ya fueron importadas en la configuración inicial.

In [ ]:
import uvicorn
import threading
import time
app = FastAPI(title="Analizador de Repositorios GitHub")

class RepositorioRequest(BaseModel):
    url_repositorio: HttpUrl  # Valida que sea una URL válida
    email_notificacion: str

class RepositorioResponse(BaseModel):
    nombre: str
    estrellas: int
    lenguaje: str
    mensaje: str

# Función que se ejecuta en background
async def analizar_issues_detallado(owner: str, repo: str, email: str):
    """
    Analiza todos los issues del repositorio y envía reporte por email.
    """
    print(f"[BACKGROUND] Iniciando análisis de issues para {owner}/{repo}...")
    
    try:
        async with httpx.AsyncClient(timeout=30.0) as cliente:
            # Obtener issues abiertos
            url_issues = f"https://api.github.com/repos/{owner}/{repo}/issues"
            respuesta = await cliente.get(url_issues)
            respuesta.raise_for_status()
            issues = respuesta.json()
            
            # Simular procesamiento pesado
            await asyncio.sleep(3)
            
            # Generar reporte
            total_issues = len(issues)
            reporte = f"""
            REPORTE DE ANÁLISIS
            ===================
            Repositorio: {owner}/{repo}
            Total de issues abiertos: {total_issues}
            Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            
            Reporte enviado a: {email}
            """
            
            print(f"[BACKGROUND] Análisis completado. Reporte:\n{reporte}")
            
    except Exception as e:
        print(f"[BACKGROUND ERROR] Error al analizar issues: {e}")

@app.post("/analizar-repositorio", response_model=RepositorioResponse)
async def analizar_repositorio(
    datos: RepositorioRequest,
    background_tasks: BackgroundTasks
):
    """
    Analiza un repositorio de GitHub.
    Responde inmediatamente con información básica.
    Genera análisis detallado en background.
    """
    # Extraer owner y repo de la URL
    # Ejemplo: https://github.com/fastapi/fastapi → owner=fastapi, repo=fastapi
    partes_url = str(datos.url_repositorio).rstrip('/').split('/')
    owner = partes_url[-2]
    repo = partes_url[-1]
    
    try:
        # Consulta rápida para información básica
        async with httpx.AsyncClient(timeout=10.0) as cliente:
            url_api = f"https://api.github.com/repos/{owner}/{repo}"
            respuesta = await cliente.get(url_api)
            respuesta.raise_for_status()
            datos_repo = respuesta.json()
        
        # Programar análisis detallado en background
        background_tasks.add_task(
            analizar_issues_detallado,
            owner,
            repo,
            datos.email_notificacion
        )
        
        # Responder inmediatamente
        return RepositorioResponse(
            nombre=datos_repo["full_name"],
            estrellas=datos_repo["stargazers_count"],
            lenguaje=datos_repo["language"] or "No especificado",
            mensaje=f"Análisis iniciado. Te enviaremos el reporte completo a {datos.email_notificacion}"
        )
        
    except httpx.HTTPStatusError:
        raise HTTPException(status_code=404, detail="Repositorio no encontrado")
    except httpx.TimeoutException:
        raise HTTPException(status_code=504, detail="GitHub API no respondió a tiempo")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error inesperado: {str(e)}")

print("Sistema completo creado. Endpoints disponibles:")
print("- POST /analizar-repositorio")

# Iniciar servidor en puerto 8005
def run_server_8005():
    uvicorn.run(app, host="127.0.0.1", port=8005, log_level="warning")

server_thread = threading.Thread(target=run_server_8005, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8005")
print(f"📚 Documentación interactiva: http://127.0.0.1:8005/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8005/redoc")


**Características del sistema:**

1. **Asincronía:** Todo el flujo usa async/await
2. **API Externa:** Consulta GitHub API con httpx
3. **Validación:** Pydantic valida que la URL sea válida
4. **Background:** Análisis pesado se hace en segundo plano
5. **Manejo de errores:** Contempla timeouts, repos inexistentes, etc.
6. **Respuesta rápida:** Usuario no espera al análisis completo

### MICRO-RETO 5: Sistema de clima con notificación

Crea un endpoint POST `/consultar-clima` que:
1. Reciba `{"ciudad": "Madrid", "email": "..."}`
2. Consulte la API: `https://api.open-meteo.com/v1/forecast?latitude=40.4168&longitude=-3.7038&current_weather=true`
3. Retorne inmediatamente la temperatura actual
4. En background: simule enviar reporte detallado por email (3 segundos de delay + print)

Nota: Usa las coordenadas de Madrid para este ejercicio (latitud=40.4168, longitud=-3.7038)

In [ ]:
import uvicorn
import threading
import time
# TODO: Tu código aquí
# Recuerda: FastAPI, BackgroundTasks, BaseModel, httpx, asyncio y TestClient ya están importados

app = FastAPI()

class ConsultaClima(BaseModel):
    ciudad: str
    email: str

# Implementa función de background y endpoint aquí

# Verificación (no modificar)
client = TestClient(app)
respuesta = client.post("/consultar-clima", json={"ciudad": "Madrid", "email": "test@test.com"})
assert respuesta.status_code == 200
datos = respuesta.json()
assert "temperatura" in datos or "temperature" in datos
print("MICRO-RETO 5 COMPLETADO")

# Iniciar servidor en puerto 8006
def run_server_8006():
    uvicorn.run(app, host="127.0.0.1", port=8006, log_level="warning")

server_thread = threading.Thread(target=run_server_8006, daemon=True)
server_thread.start()
time.sleep(2)

print("✅ Servidor iniciado en http://127.0.0.1:8006")
print(f"📚 Documentación interactiva: http://127.0.0.1:8006/docs")
print(f"📖 Documentación alternativa: http://127.0.0.1:8006/redoc")
